In [1]:
#importing the neccesary libraries

import pandas as pd
import geopandas as gpd
import plotly as plt
import folium 
import matplotlib.pyplot as mplt
import contextily as ctx
from shapely import wkt
from shapely.wkt import loads
import plotly.express as px
import osmnx as ox

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# Importing Block Group Data and Shapefile
CA_BlockGroup_2019 = gpd.read_file('../Data/Employment Block Groups ACS 2019.csv')
CA_BlockGroup_2010 = gpd.read_file('../Data/Employment Block Groups ACS 2010.csv')
CA_BlockGroup_Shapes = gpd.read_file('../Data/CA Block Group 2019/ca_blockgroup_2019.shp')
Ontario_boundary = gpd.read_file('../Data/Ontario Boundary/Ontario Boundary.shp')
                                     
Ontario_boundary = Ontario_boundary.to_crs(epsg=4326)
CA_BlockGroup_Shapes = CA_BlockGroup_Shapes.to_crs(epsg=4326)

In [3]:
#Keep only San Bernandino County Block Groups
CA_BlockGroup_Shapes = CA_BlockGroup_Shapes[CA_BlockGroup_Shapes['COUNTYFP'] == '071']

In [4]:
#keep only population, population density, # of warehouse workers, and median household income
columns_to_keep = ['Geo_FIPS',
                   'Geo_GEOID',
                   'SE_A17007B_014',
                   'SE_A00002_001',
                   'SE_A00002_002',
                   'SE_A17007A_014',
                   'SE_A14006_001',
                  ]
# create new dataframe with the selected columns
CA_BlockGroup_2010 = CA_BlockGroup_2010[columns_to_keep]
CA_BlockGroup_2019 = CA_BlockGroup_2019[columns_to_keep]

In [6]:
#Rename Columns
columns19 = list(CA_BlockGroup_2010)
columns10 = list(CA_BlockGroup_2019)

print(columns19)
print(columns10)

CA_BlockGroup_2010.columns = ['GEOID',
                   'Geo_GEOID',
                   'Female Transportation and Material Moving Occupations 2010',
                   'Total Population 2010',
                   'Population Density (Per Sq. Mile) 2010',
                   'Male Transportation and Material Moving Occupations 2010',
                   'Median Household Income (In 2019 Inflation Adjusted Dollars) 2010'
                  ]
CA_BlockGroup_2019.columns = ['GEOID',
                   'Geo_GEOID',
                   'Female Transportation and Material Moving Occupations 2019',
                   'Total Population 2019',
                   'Population Density (Per Sq. Mile) 2019',
                   'Male Transportation and Material Moving Occupations 2019',
                   'Median Household Income (In 2019 Inflation Adjusted Dollars) 2019'
                  ]

#need to multiply median household income of 2010 by 16% According to the Bureau of Labor Statistics
CA_BlockGroup_2010['Median Household Income (In 2019 Inflation Adjusted Dollars) 2010'].replace('', np.nan, inplace=True)
CA_BlockGroup_2019['Median Household Income (In 2019 Inflation Adjusted Dollars) 2019'].replace('', np.nan, inplace=True)
CA_BlockGroup_2010 = CA_BlockGroup_2010.dropna(subset=['Median Household Income (In 2019 Inflation Adjusted Dollars) 2010'])
CA_BlockGroup_2019 = CA_BlockGroup_2019.dropna(subset=['Median Household Income (In 2019 Inflation Adjusted Dollars) 2019'])
data_types_dict = {'Median Household Income (In 2019 Inflation Adjusted Dollars) 2010': float}
CA_BlockGroup_2010 = CA_BlockGroup_2010.astype(data_types_dict)
data_types_dict = {'Median Household Income (In 2019 Inflation Adjusted Dollars) 2019': float}
CA_BlockGroup_2019 = CA_BlockGroup_2019.astype(data_types_dict)
CA_BlockGroup_2010['Median Household Income (In 2019 Inflation Adjusted Dollars) 2010'] = CA_BlockGroup_2010['Median Household Income (In 2019 Inflation Adjusted Dollars) 2010'] * 1.16

['GEOID', 'Geo_GEOID', 'Female Transportation and Material Moving Occupations 2010', 'Total Population 2010', 'Population Density (Per Sq. Mile) 2010', 'Male Transportation and Material Moving Occupations 2010', 'Median Household Income (In 2019 Inflation Adjusted Dollars) 2010']
['GEOID', 'Geo_GEOID', 'Female Transportation and Material Moving Occupations 2019', 'Total Population 2019', 'Population Density (Per Sq. Mile) 2019', 'Male Transportation and Material Moving Occupations 2019', 'Median Household Income (In 2019 Inflation Adjusted Dollars) 2019']


NameError: name 'np' is not defined

In [ ]:
#merge datasets
Temp_Shape=CA_BlockGroup_Shapes.merge(CA_BlockGroup_2019,on="GEOID")
SB_County_Block_Group_Employment=Temp_Shape.merge(CA_BlockGroup_2010,on="GEOID")
data_types_dict = {
    'Female Transportation and Material Moving Occupations 2010': float,
    'Female Transportation and Material Moving Occupations 2019' : float,
    'Male Transportation and Material Moving Occupations 2019' : float,
    'Male Transportation and Material Moving Occupations 2010' : float,
}
SB_County_Block_Group_Employment = SB_County_Block_Group_Employment.astype(data_types_dict)
SB_County_Block_Group_Employment['Change in Median Household Income from 2010 to 2019 (2019 dollars)'] = SB_County_Block_Group_Employment['Median Household Income (In 2019 Inflation Adjusted Dollars) 2019']-SB_County_Block_Group_Employment['Median Household Income (In 2019 Inflation Adjusted Dollars) 2010']
SB_County_Block_Group_Employment['Transportation and Material Moving Occupations 2019'] = SB_County_Block_Group_Employment['Male Transportation and Material Moving Occupations 2019']+SB_County_Block_Group_Employment['Female Transportation and Material Moving Occupations 2019']
SB_County_Block_Group_Employment['Transportation and Material Moving Occupations 2010'] = SB_County_Block_Group_Employment['Male Transportation and Material Moving Occupations 2010']+SB_County_Block_Group_Employment['Female Transportation and Material Moving Occupations 2010']
SB_County_Block_Group_Employment['Change in Warehouse Employment from 2010 to 2019'] = SB_County_Block_Group_Employment['Transportation and Material Moving Occupations 2019']-SB_County_Block_Group_Employment['Transportation and Material Moving Occupations 2010']

In [ ]:
#create Shapefile
#SB_County_Block_Group_Employment.to_csv("SB_County_Block_Group_Employment_2010_2019.csv")
#SB_County_Block_Group_Employment.to_file("SB_County_Block_Group_Employment_2010_2019.shp")